# Using Opik with Pydantic AI

Opik integrates with [Pydantic AI](https://ai.pydantic.dev/) to provide a simple way to log your agent calls.


## Creating an account on Comet.com

[Comet](https://www.comet.com/site?from=llm&utm_source=opik&utm_medium=colab&utm_content=openai&utm_campaign=opik) provides a hosted version of the Opik platform, [simply create an account](https://www.comet.com/signup?from=llm&utm_source=opik&utm_medium=colab&utm_content=aisuite&utm_campaign=opik) and grab you API Key.

> You can also run the Opik platform locally, see the [installation guide](https://www.comet.com/docs/opik/self-host/overview/?from=llm&utm_source=opik&utm_medium=colab&utm_content=aisuite&utm_campaign=opik) for more information.

## Setting up the logging

Pydantic AI uses the logfire library to log traces to Opik. Before logging our agent calls, we will configure the integration
by installing the required libraries and setting the correct environment variables:

In [15]:
import os

# Install the required libraries
%pip install --upgrade --quiet pydantic-ai logfire 'logfire[httpx]' nest_asyncio

# Configure the logging to Opik Cloud
os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = (
    "https://www.comet.com/opik/api/v1/private/otel"
)
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = (
    "Authorization=your-api-key,Comet-Workspace=default"  # Make sure to replace your API key
)

# If you are using a self-hosted instance, you can use:
# os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "http://localhost:5173/api/v1/private/otel"


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Now that everything is configured correctly, we can enable the logging:

In [20]:
import logfire

logfire.configure(
    send_to_logfire=False,
)
logfire.instrument_httpx(capture_all=True)

Attempting to instrument while already instrumented


## Logging your first trace

Before we log our first trace, we are going to configure the environment and ensure
the code runs in a Notebook

In [25]:
import nest_asyncio
import os
import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

nest_asyncio.apply()

In [26]:
import os

from pydantic_ai import Agent

agent = Agent(
    "openai:gpt-4o",
    system_prompt="Be concise, reply with one sentence.",
)

result = agent.run_sync('Where does "hello world" come from?')
print(result.data)

11:57:41.825 POST api.openai.com/v1/chat/completions
11:57:43.481 Reading response body
"Hello, World!" originates from the book "The C Programming Language" by Brian Kernighan and Dennis Ritchie, often used as a simple program to illustrate basic syntax in programming.


![pydantic-ai Integration](https://raw.githubusercontent.com/comet-ml/opik/main/apps/opik-documentation/documentation/fern/img/cookbook/pydantic-ai_trace_cookbook.png)

*Note:* This screenshot was taken for a more complex agent but you should see a very trace.